<a href="https://colab.research.google.com/github/Kira1108/langchain-experiments/blob/main/LangChain_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from IPython.display import clear_output
!pip install langchain
!pip install openai
!pip install git+https://github.com/Kira1108/simple-chat.git
clear_output()

In [3]:
try:
    from simple_chat.printer import MarkdownPrinter
    printer = MarkdownPrinter()
except:
    printer = print

In [4]:
import os
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
import sqlite3
import os
os.environ['OPENAI_API_KEY'] = ''

### 对于llm，掌握openai就够了，其他的也没钱部署

In [5]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

chatgpt = ChatOpenAI()
davinci = OpenAI()

print("llm model name:", davinci.model_name)
print("chatgpt model name: ",chatgpt.model_name)

llm model name: text-davinci-003
chatgpt model name:  gpt-3.5-turbo


### PromptTemplate

- 带参数的Prompt
- 不带参数的Prompt
- 有花括号的Prompt String

In [6]:
from langchain import PromptTemplate


template = """
I want you to act as a naming consultant for new companies.

Here are some examples of good company names:

- search engine, Google
- social media, Facebook
- video sharing, YouTube

The name should be short, catchy and easy to remember.

What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

printer(prompt.format(product = "headphone"))


I want you to act as a naming consultant for new companies.

Here are some examples of good company names:

- search engine, Google
- social media, Facebook
- video sharing, YouTube

The name should be short, catchy and easy to remember.

What is a good name for a company that makes headphone?


In [7]:
template = """
This is a custom template.
In this template, I have a variable {var1}
and another variable {var2}

In order to use a curly brace, I will have to use
{{some_text}}
"""

prompt = PromptTemplate(
    input_variables=["var1",'var2'],
    template=template,
)

printer(prompt.format(var1 = 3, var2 = 4))


This is a custom template.
In this template, I have a variable 3
and another variable 4

In order to use a curly brace, I will have to use
{some_text}


In [8]:
template = "This is a prompt with no variable"

prompt = PromptTemplate(input_variables = [], template = template)
printer(prompt.format())

This is a prompt with no variable

### FewShot Template

 - 首先你要构造样本提示语 数据样本 - `example`是`list of json`，用`example_formatter_template`构造一个`PromptTemplate`， 这俩东西在FewShotTemplate里面组装
 - 然后你要构造小样本学习的提示语 - `FewShotPromptTemplate` 这里需要知道小样本学习的流程。
 1. 输入任务
 2. 输入样本
 3. 接收用户输入，等待模型输出

 ```python
 我今天需要完成一个牛逼的任务 # 这个是prefix,当作任务描述就行了

 输入=白， 结果 = 黑 # 这儿是小样本，exmaple插入example_formatter_template
 输入=高， 结果 = 矮

 输入=傻   结果 = # 这是suffix, input_variables = 傻， 插入 suffix
 ```

 分三段写，第一段写任务，第二段写样本，第三段写输出

In [9]:
from langchain import PromptTemplate, FewShotPromptTemplate


# First, create the list of few shot examples.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="",
)

# We can now generate a prompt using the `format` method.
print(few_shot_prompt.format(input="big"))

Give the antonym of every input
Word: happy
Antonym: sad


Word: tall
Antonym: short

Word: big
Antonym:


### Example Selector

In [10]:
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""

from langchain.prompts.example_selector import LengthBasedExampleSelector


# These are a lot of examples of a pretend task of creating antonyms.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

# We'll use the `LengthBasedExampleSelector` to select the examples.
example_selector = LengthBasedExampleSelector(
    # These are the examples is has available to choose from.
    examples=examples, 
    # This is the PromptTemplate being used to format the examples.
    example_prompt=example_prompt, 
    # This is the maximum length that the formatted examples should be.
    # Length is measured by the get_text_length function below.
    max_length=25,
)

# We can now use the `example_selector` to create a `FewShotPromptTemplate`.
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n",
)

# We can now generate a prompt using the `format` method.
printer(dynamic_prompt.format(input="big"))

Give the antonym of every input

Word: happy
Antonym: sad



Word: tall
Antonym: short



Word: energetic
Antonym: lethargic


Word: big
Antonym:

In [11]:
printer(dynamic_prompt.format(input="big small and niubi and there are alog of words here in the input string, cool"))

Give the antonym of every input

Word: happy
Antonym: sad


Word: big small and niubi and there are alog of words here in the input string, cool
Antonym:

In [12]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "Who lived longer, Muhammad Ali or Alan Turing?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
  },
  {
    "question": "When was the founder of craigslist born?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
  },
  {
    "question": "Who was the maternal grandfather of George Washington?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
  },
  {
    "question": "Are both the directors of Jaws and Casino Royale from the same country?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
  }
]


example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

for i in range(3):
    printer(example_prompt.format(**examples[i]))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


In [13]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Question: {input}", 
    input_variables=["input"]
)

printer(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


Question: Are both the directors of Jaws and Casino Royale from the same country?

Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No


Question: Who was the father of Mary Ball Washington?

### Prompt Subclass

In [43]:
import inspect
import textwrap

def get_source_code(py_func):
    return inspect.getsource(py_func)


from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator


FUNCTION_PROMPT = """
Given the function name and source code, generate an English language explanation of the function.
Function name : {function_name}
Source Code:
```python
{source_code}
```
Explaination:
""".strip()

class FunctionExaplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """
    This is very stupid if no documentation is provided.
    In this case, you only pass a input_variables list to the constructor of the prompt
    The input_variables lists all parameters required to format the prompt.
    """
    @validator("input_variables")
    def validate_input_variables(cls, v):
        if len(v) != 1 or "py_func" not in v:
            raise ValueError("py_func must be the only input_variable")
        return v
    
    # when calling this function, you have to provide the necessary parameters defined in input_variables
    def format(self, **kwargs) -> str:
        fn = kwargs['py_func']
        source_code = get_source_code(fn)
        return FUNCTION_PROMPT.format(function_name = fn.__name__, source_code = source_code)

    def _prompt_type(self):
        return "function-explainer"

def somefunction():
    pass

prompt_template = FunctionExaplainerPromptTemplate(input_variables = ['py_func'])

print(prompt_template.format(py_func = somefunction))

Given the function name and source code, generate an English language explanation of the function.
Function name : somefunction
Source Code:
```python
def somefunction():
    pass

```
Explaination:


### Partial prompts (Very Silly)

In [49]:
from langchain.prompts import PromptTemplate

p = PromptTemplate(template = "{foo}-{bar}", input_variables = ['foo','bar'])

# patialize the prompt, set foo = 3
p = p.partial(foo = 3)

# format the prompt bar = 12003
p.format(bar=  12003)

'3-12003'

Partial with a function

In [52]:
from datetime import datetime

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

p = PromptTemplate(template = "{content} - {time}", input_variables = ['content','time'])
p = p.partial(time= _get_datetime)
p.format(content = 'This is content')

'This is content - 03/31/2023, 14:35:54'